# Classification Decision Tree

## Import Modules

In [ ]:
import numpy as np
import pandas as pd

## The Dataset

* We use the classic two-moons dataset, consisting of two interleaving half circles with added noise.

* The dataset is conveniently built-in to `Scikit-learn` and accessible via  `make_moons`, which returns a data matrix `X ` and a label vector `y`. 
* We can treat the dataset as a complete training set as we eventually use cross-validation to estimate the test error.

In [ ]:
'''
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

data = make_moons(n_samples=10000, shuffle=True, noise=0.4, random_state=42)
X,Y=data
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

plt.scatter(X[:,0],X[:,1], c=Y)
plt.show()
'''

'\nfrom sklearn.datasets import make_moons\nimport matplotlib.pyplot as plt\n\ndata = make_moons(n_samples=10000, shuffle=True, noise=0.4, random_state=42)\nX,Y=data\n#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)\n\nplt.scatter(X[:,0],X[:,1], c=Y)\nplt.show()\n'

In [ ]:
import seaborn as sns
data = pd.read_csv('/content/drive/MyDrive/Ensemble Learning/Tweets/all_features.csv', index_col=0)

In [ ]:
data.head()

,tweet_text,cyberbullying_type,characters per tweet,words_per_tweet,nb_upper,nb_lower,nb_capitalized,mixed_upper_lower_not_capitalized,nb_len_1,nb_len_2,...,muslim,gay,round,good,radical,bad,mkr,rape,stupid,lot
0,"in other words #katandandre, your food was cra...",not_cyberbullying,61,9,0,8,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,why is #aussietv so white? #mkr #theblock #ima...,not_cyberbullying,115,14,1,9,1,0,0,2,...,0,0,0,0,0,0,1,0,0,0
2,@xochitlsuckkks a classy whore? or more red ve...,not_cyberbullying,60,9,0,7,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,"@jason_gio meh. :p thanks for the heads up, b...",not_cyberbullying,103,18,1,16,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,@rudhoeenglish this is an isis account pretend...,not_cyberbullying,103,18,1,12,4,0,1,6,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.columns

Index(['tweet_text', 'cyberbullying_type', 'characters per tweet',
       'words_per_tweet', 'nb_upper', 'nb_lower', 'nb_capitalized',
       'mixed_upper_lower_not_capitalized', 'nb_len_1', 'nb_len_2',
       ...
       'muslim', 'gay', 'round', 'good', 'radical', 'bad', 'mkr', 'rape',
       'stupid', 'lot'],
      dtype='object', length=179)

## Importing data

In [ ]:
data.dtypes

0         float64
1         float64
2         float64
3         float64
4         float64
           ...   
bad         int64
mkr         int64
rape        int64
stupid      int64
lot         int64
Length: 1179, dtype: object

In [ ]:
import seaborn as sns

sns.set(style="ticks", color_codes=True)
sns.pairplot(data,  hue="species")

ValueError: ignored

In [ ]:
# Correlation map
sns.heatmap(data.corr())

ValueError: ignored

## Node class

A small class just containing a constructor, which will basically describe the nodes of our tree. 

In [ ]:
class Node():
    
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):      
        
        # A tree has two types of nodes essentially: decision nodes and leaf nodes.
        
        # Decision nodes:
        self.feature_index = feature_index
        self.threshold = threshold # threshold value for the specific feature 
        
        # right and left are for accessing the left child and right child, in order to move from parent to child
        self.left = left
        self.right = right
        self.info_gain = info_gain # stores the information gain of the split 
        
        # Leaf node
        self.value = value # value is the majority class of the leaf node - will help us predict the class of a new data point if it ends up in this particular leaf node

## Tree class

The actual algorithm, which will contain a tree building part, entropy calculation etc.

In [ ]:
class DecisionTreeClassifier():
    
    # first, we define a constructor   
    def __init__(self, min_samples_split=2, max_depth=2): # if n.of samples < min n. of samples, then won't split further
        
        ''' constructor '''
        
        # Initialize the root of the tree
        self.root = None # to travel through the tree
        
        # Stopping conditions 
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    # most important function: it builds the binary tree using recursion
    def build_tree(self, dataset, curr_depth=0):
        
        ''' building the tree '''
        
        # splits the features and targets into two separate variables 
        X, Y = dataset[:,:-1], dataset[:,-1]
        
        # extracts the number of samples and features 
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met (checks whether n. of samples < min n. of samples)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # if these two conditions are not met it will not split further 
            
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            ''' check if information gain is positive
            n.b. info gain = 0 means we are splitting a node which is already pure, i.e. the node consists of only 
            one type of class, and that does not make any sesnse
            '''
            
            if best_split["info_gain"]>0:
                
                # recursive part (left): we call build_tree
                
                '''
                will first create left subtree and then once it has reached the leaf node, 
                it will create right subtree 
                '''
                
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
               
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],   # best_split is a dictionary
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split 
        returns a dictionary, indeed best_split dicsionary is defined at the beginning and max info gain is initializeed 
        as negatve infinity, since we want to maximise the information gain, and to find that we need to use a  number
        bigger that any other number '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
         
        
        '''
        Here, we will loop over all teh features, through all the possible treshold values.
        N.b. it would not make sense to iterate over every possible real number. 
        Indeed, we will iterate through every possible value of a fetaure 
        that we have encountered on our dataet: np.unique  
        '''
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # we split the dataset based on the current feature index and the current treshold 
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if child not null - we have to make sure they are not empty
                
                #then we extract the target values (Y)      
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain: 
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' 
        function to split the data 
        '''
        
        # left side: i pass those feature values smaller than the treshold 
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        
        ''' function to compute information gain 
            this function basically substracts the comnbined information 
            of the child node from the parent node.
            
            weights: relative sizes of the child with respect to the parent.
            '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        
        ''' function to compute leaf node.
        
        For regression, leaf node is computed by the mean of the values in the predictor column, while
        for classificaiton, the leaf node is computed by majority rule, i.e. the majority class present
        in a particular node. 
        
        '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        
        ''' function to train the tree '''
        # concatenate x and y to create the dataset
        dataset = np.concatenate((X, Y), axis=1)
        
        # call the build tree function
        #root node will be returned by the build tree fct and it will be stored in self.root
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        ''' function to predict new dataset '''
        ''' will take a new dataset (matrix of feature) and return the corresponding predictions '''
        preditions = [self.make_prediction(x, self.root) for x in X]
        
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point:
        
        takes a node as parameter -> we are passing the root node first 
        
        if node.value is not none, meaning if the node is a leaf node, then it returns the value
        if it is not a leaf node, then it will extract the feature value of our new 
        data point and the given feature index 
        '''
        
        if tree.value!=None: return tree.value
    
        
        feature_val = x[tree.feature_index]
        
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

# Training and testing

## Train-Test split

In [ ]:
df = data.drop('tweet_text', 1)
df.head()

In [ ]:
df_1 = df[1:101]
df_1

,cyberbullying_type,characters per tweet,words_per_tweet,nb_upper,nb_lower,nb_capitalized,mixed_upper_lower_not_capitalized,nb_len_1,nb_len_2,nb_len_3,...,muslim,gay,round,good,radical,bad,mkr,rape,stupid,lot
1,not_cyberbullying,115,14,1,9,1,0,0,2,1,...,0,0,0,0,0,0,1,0,0,0
2,not_cyberbullying,60,9,0,7,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,not_cyberbullying,103,18,1,16,0,0,0,2,6,...,0,0,0,0,0,0,0,0,0,0
4,not_cyberbullying,103,18,1,12,4,0,1,6,1,...,0,0,0,0,0,0,0,0,0,0
5,not_cyberbullying,131,23,0,20,1,0,0,7,4,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,not_cyberbullying,93,15,1,12,1,0,2,1,4,...,0,0,0,0,0,0,0,0,0,0
97,not_cyberbullying,41,10,2,5,3,0,2,0,4,...,0,0,0,0,0,0,0,0,0,0
98,not_cyberbullying,84,14,0,13,1,0,1,0,4,...,0,0,0,0,0,0,0,0,0,0
99,not_cyberbullying,135,20,2,14,2,0,0,2,3,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
X = pd.DataFrame(df.loc[:, df.columns != 'cyberbullying_type'])
Y = pd.DataFrame(df['cyberbullying_type'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [ ]:
Y.columns

Index(['cyberbullying_type'], dtype='object')

## Fit the model and visualize

In [38]:
classifier = DecisionTreeClassifier(max_depth=5)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_156 <= 0 ? 0.14282857844333652
 left:X_135 <= 0 ? 0.12028680076482312
  left:X_168 <= 0 ? 0.08925087490801775
    left:X_0 <= 144 ? 0.1059043011343731
        left:X_89 <= 0 ? 0.022006478305033883
                left:X_121 <= 0 ? 0.021088785424614676
                                left:other_cyberbullying
                                right:religion
                right:X_3 <= 6 ? 0.019134738859332645
                                left:religion
                                right:religion
        right:X_104 <= 0 ? 0.08743570257362765
                left:X_99 <= 0 ? 0.09246906230832419
                                left:religion
                                right:gender
                right:X_164 <= 0 ? 0.1821618458581349
                                left:ethnicity
                                right:religion
    right:X_174 <= 0 ? 0.03362557253308643
        left:X_126 <= 0 ? 0.10669044759953861
                left:X_26 <= 8 ? 0.10131917493490472
              

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Vizualize

## Test the model

In [ ]:
Y_pred = classifier.predict(X_test) 

from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9333333333333333